In [1]:
pip show transformers

Name: transformers
Version: 4.38.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [2]:
!pip install transformers datasets sentencepiece

In [3]:
from transformers import T5ForConditionalGeneration, RobertaTokenizer, Trainer,TrainingArguments
from datasets import load_dataset
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback
from transformers import Seq2SeqTrainer

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
# Load your dataset
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/preprocessed_data (1).csv")

In [6]:
# Split data into training and validation sets (adjust test_size as needed)
test_size = 0.2
X_train, X_val, y_train, y_val = train_test_split(data["code"],
                                                  data["summary"],
                                                  test_size=test_size,
                                                  random_state=42)
train_df = pd.DataFrame({"code": X_train, "summary": y_train})
val_df = pd.DataFrame({"code": X_val, "summary": y_val})


In [7]:
model_name = "Salesforce/codet5-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
# from transformers import TrainingArguments, EarlyStoppingCallback
# generation_config = {
#     "max_length": 128,
#     "temperature": 1.0,
#     "top_k": 50,
# }
training_args = TrainingArguments(
    output_dir="./results",  # Adjust the output directory
    per_device_train_batch_size=8,  # Adjust batch size as needed
    save_steps=5000,
    num_train_epochs=5,  # Adjust training epochs
    logging_steps=500,
    evaluation_strategy="epoch",
    metric_for_best_model="bleu",

)
# Define EarlyStoppingCallback with desired patience
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)
# Define custom compute metrics function for BLEU score
def compute_metrics(pred):
    label_ids, preds = pred
    pred_list = tokenizer.batch_decode(preds, skip_special_tokens=True)
    label_list = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    bleu_score = sentence_bleu([[ref] for ref in label_list], pred_list)
    return {"bleu": bleu_score}


In [9]:
pip cache purge

Files removed: 1


In [ ]:
generation_config = {
    "max_length": 128,
    "temperature": 1.0,
    "top_k": 50,
}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_df,
    eval_dataset=val_df,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback],
    generation_config=generation_config,

)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("./fine-tuned_codet5")  # Adjust the save path
tokenizer.save_pretrained("./fine-tuned_codet5")
